In [1]:
import nb_utils

Moving cwd to /store/CIA/js2164/repos/diffusion/score_sde_pytorch


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from mle import bootstrap_intrinsic_dim_scale_interval as mle

/store/CIA/js2164/conda_envs/score_sde/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Import R packages
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr
r_base = importr('base')
intdimr = importr('intrinsicDimension')

In [4]:
from lightning_data_modules import HaarDecomposedDataset, ImageDatasets, PairedDataset, SyntheticDataset, SyntheticPairedDataset, Synthetic1DConditionalDataset, SyntheticTimeSeries, SRDataset, SRFLOWDataset, KSphereDataset, MammothDataset, LineDataset, GanDataset #needed for datamodule registration

#from configs.mammoth.vesde import get_config
#from configs.ksphere.N_1.uniform import get_config
#from configs.line.vesde import get_config
#from configs.ksphere.N_1.non_uniform_1 import get_config
#from configs.ksphere.N_1.non_uniform_075 import get_config
#from configs.ksphere.N_1.non_uniform_05 import get_config
from configs.utils import read_config

config = read_config('configs/fixedgaussiansmanifold/10.py')

In [5]:
from lightning_data_modules.utils import create_lightning_datamodule
DataModule = create_lightning_datamodule(config)
DataModule.setup()
train_dataloader = DataModule.train_dataloader()

In [6]:
X=[]
for _, x in enumerate(train_dataloader):
    X.append(x.view(x.shape[0],-1))
X = torch.cat(X, dim=0).numpy()
X = pd.DataFrame(X)

In [7]:
print(X.shape)
X.head()

(400000, 1024)


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.163414,0.219268,0.172705,0.087923,0.040728,0.026780,0.021884,0.017580,0.013557,0.010892,...,1.386236e-03,1.169709e-03,9.393941e-04,7.172892e-04,5.197694e-04,3.562091e-04,2.293351e-04,1.367738e-04,7.308478e-05,0.000032
1,0.291385,0.321956,0.296765,0.229388,0.150558,0.086805,0.048435,0.032603,0.032432,0.042975,...,1.585719e-05,9.041454e-06,4.941475e-06,2.586065e-06,1.292872e-06,6.139327e-07,2.728862e-07,1.088855e-07,3.334648e-08,0.000000
2,0.221626,0.240306,0.233038,0.203508,0.161930,0.119619,0.084314,0.058766,0.042237,0.032923,...,2.503101e-05,2.245073e-05,1.873668e-05,1.451978e-05,1.040986e-05,6.853618e-06,4.074396e-06,2.092036e-06,7.912957e-07,0.000000
3,0.166387,0.171150,0.167155,0.154942,0.136198,0.113498,0.089847,0.068133,0.050737,0.039529,...,2.415670e-09,1.941207e-09,1.348973e-09,8.102046e-10,4.202705e-10,1.878954e-10,7.191578e-11,2.296552e-11,5.389860e-12,0.000000
4,0.298547,0.318099,0.315551,0.292095,0.253061,0.205980,0.158263,0.115461,0.080620,0.054654,...,1.185406e-06,3.937646e-07,1.245425e-07,3.750327e-08,1.074834e-08,2.927867e-09,7.539127e-10,1.791338e-10,3.458217e-11,0.000000


# MLE

In [14]:
k = 5
mle_result = mle(X,  k1=k, k2=k, average=True)[0]

KeyboardInterrupt: 

In [13]:
mle_result

NameError: name 'mle_result' is not defined

# MLE R

In [8]:
data_r = X.to_numpy()
dim = X.shape[1]
k=5
#mle = intdimr.maxLikPointwiseDimEst(data_r, k=5).rx2('dim.est')
mle = intdimr.maxLikGlobalDimEst(data_r, k=k).rx2('dim.est')

In [ ]:
mle

array([32.79388876])

# Local PCA

In [28]:
data_r = X
dim = X.shape[1]
lpca = intdimr.pcaLocalDimEst(data_r, 'FO').rx2('dim.est')

In [29]:
lpca

array([72], dtype=int32)

In [9]:
from sklearn.decomposition import PCA
data_r = X.to_numpy().astype(np.float64)
pca = PCA(n_components='mle')
pca.fit(data_r)
pca.n_components_


99

In [46]:
data_r + 1e-12

array([[1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       ...,
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12]],
      dtype=float32)

In [49]:
data_r

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)